# MonteCarlo Prediction

Given the environment description (policy and transition probabilities ) prediction is the computations done to evaluate the value function when we follow a given policy. This demo intends to evaluate policy using first visit montaecarlo estimation.

In [1]:
import numpy as np
from gridworld import GridWorld

np.set_printoptions(precision=3,suppress=True)

Return computation function takes as input a list containing a tuple (state, reward) and the discount factor gamma, the output is a value representing the return from that state.

In [2]:
def get_return(state_list, gamma):
    counter = 0
    return_value = 0
    for visit in state_list:
        reward = visit[1]
        return_value += reward * np.power(gamma, counter)
        counter += 1
    return return_value

1.Lets create a grid world, the states marked 1 are terminal state and those marked -1 contain obstacles. <br>
2.The agent receives a reward of -0.04 for every move from non-terminal states <br>
3.The  actions are UP(0), RIGHT(1), DOWN(2) and LEFT(3)

In [3]:
env = GridWorld(3, 4)

#Define the state matrix
state_matrix = np.zeros((3,4))
state_matrix[0, 3] = 1
state_matrix[1, 3] = 1
state_matrix[1, 1] = -1
print("State Matrix:")
print(state_matrix)

State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]


Lets also define rewards.

In [4]:
 #Define the reward matrix
reward_matrix = np.full((3,4), -0.04)
reward_matrix[0, 3] = 1
reward_matrix[1, 3] = -1
print("Reward Matrix:")
print(reward_matrix)

Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]


Lets define the transition probabilities for four possible actions. 

In [5]:
 #Define the transition matrix
transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                              [0.1, 0.8, 0.1, 0.0],
                              [0.0, 0.1, 0.8, 0.1],
                              [0.1, 0.0, 0.1, 0.8]])

Since this grid is designed by us, we know the best policy to follow. Lets define it now and evaluate it later.

In [6]:
policy_matrix = np.array([[1,      1,  1,  -1],
                              [0, np.NaN,  0,  -1],
                              [0,      3,  3,   3]])
policy_matrix

array([[ 1.,  1.,  1., -1.],
       [ 0., nan,  0., -1.],
       [ 0.,  3.,  3.,  3.]])

In [7]:

env = GridWorld(3, 4)
env.reset()
env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)

In [8]:
env.render()

 -  -  -  * 
 -  #  -  * 
 ○  -  -  - 



In [9]:
value_matrix = np.zeros((3,4))
#init with 1.0e-10 to avoid division by zero
running_mean_matrix = np.full((3,4), 1.0e-10) 
gamma = 0.999
tot_epoch = 10000
print_epoch = 1000


for epoch in range(tot_epoch):
    #Starting a new episode
    episode_list = list()
    #Reset and return the first observation and reward
    observation = env.reset(exploring_starts=True)
    for _ in range(1000):
        #Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        #Move one step in the environment and get obs and reward
        observation, reward, done = env.step(action)
        #Append the visit in the episode list
        episode_list.append((observation, reward))
        if done: break
    #The episode is finished, now estimating the value function
    counter = 0
    #Checkup to identify if it is the first visit to a state
    checkup_matrix = np.zeros((3,4))
    #This cycle is the implementation of First-Visit MC.
    #For each state stored in the episode list check if it
    #is the first visit then estimate the return.
    for visit in episode_list:
        observation = visit[0]
        row = observation[0]
        col = observation[1]
        reward = visit[1]
        if(checkup_matrix[row, col] == 0):
            return_value = get_return(episode_list[counter:], gamma)
            running_mean_matrix[row, col] += 1
            value_matrix[row, col] += return_value
            checkup_matrix[row, col] = 1
        counter += 1
    if(epoch % print_epoch == 0):
        print("")
        print("value matrix after " + str(epoch+1) + " iterations:") 
        print(value_matrix / running_mean_matrix)
#Time to check the value matrix obtained
print("value matrix after " + str(tot_epoch) + " iterations:")
print(value_matrix / running_mean_matrix)



value matrix after 1 iterations:
[[0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

value matrix after 1001 iterations:
[[ 0.817  0.875  0.924  1.   ]
 [ 0.768  0.     0.708 -1.   ]
 [ 0.716  0.662  0.644  0.518]]

value matrix after 2001 iterations:
[[ 0.816  0.876  0.927  1.   ]
 [ 0.769  0.     0.728 -1.   ]
 [ 0.716  0.665  0.643  0.459]]

value matrix after 3001 iterations:
[[ 0.813  0.874  0.922  1.   ]
 [ 0.766  0.     0.699 -1.   ]
 [ 0.711  0.656  0.625  0.504]]

value matrix after 4001 iterations:
[[ 0.813  0.873  0.921  1.   ]
 [ 0.763  0.     0.703 -1.   ]
 [ 0.71   0.658  0.621  0.503]]

value matrix after 5001 iterations:
[[ 0.811  0.871  0.921  1.   ]
 [ 0.761  0.     0.699 -1.   ]
 [ 0.707  0.655  0.617  0.439]]

value matrix after 6001 iterations:
[[ 0.812  0.872  0.922  1.   ]
 [ 0.761  0.     0.7   -1.   ]
 [ 0.708  0.657  0.617  0.446]]

value matrix after 7001 iterations:
[[ 0.812  0.872  0.922  1.   ]
 [ 0.761  0.     0.695 -1.   ]
 [ 0.709  0.659  0.624  0.448]]

va